In [1]:
import numpy as np
import time
import cv2

In [2]:
INPUT_FILE='C:/Users/Himanshu Jawale/Pedestrain_Detection/yash_1.jpg'
OUTPUT_FILE='predicted.jpg'
LABELS_FILE='C:/Users/Himanshu Jawale/Pedestrain_Detection/darknet-master/data/coco.names'
CONFIG_FILE='C:/Users/Himanshu Jawale/Pedestrain_Detection/darknet-master/cfg/yolov3.cfg'
WEIGHTS_FILE='yolov3.weights'

In [3]:
CONFIDENCE_THRESHOLD=0.3

In [4]:
LABELS = open(LABELS_FILE).read().strip().split("\n")

In [5]:
np.random.seed(4)

In [6]:
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")

In [7]:
net = cv2.dnn.readNetFromDarknet(CONFIG_FILE, WEIGHTS_FILE)


In [8]:
image = cv2.imread(INPUT_FILE)

In [9]:
(H, W) = image.shape[:2]

# determine only the *output* layer names that we need from YOLO


In [10]:
ln = net.getLayerNames()

In [11]:
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]


In [12]:
blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
	swapRB=True, crop=False)

In [13]:
net.setInput(blob)

In [14]:
start = time.time()

In [15]:
layerOutputs = net.forward(ln)

In [16]:
end = time.time()

In [17]:
print("[INFO] YOLO took {:.6f} seconds".format(end - start))


[INFO] YOLO took 1.116017 seconds


# initialize our lists of detected bounding boxes, confidences, and
# class IDs, respectively

In [18]:
boxes = []
confidences = []
classIDs = []

In [19]:
for output in layerOutputs:
	# loop over each of the detections
	for detection in output:
		# extract the class ID and confidence (i.e., probability) of
		# the current object detection
		scores = detection[5:]
		classID = np.argmax(scores)
		confidence = scores[classID]

		# filter out weak predictions by ensuring the detected
		# probability is greater than the minimum probability
		if confidence > CONFIDENCE_THRESHOLD:
			# scale the bounding box coordinates back relative to the
			# size of the image, keeping in mind that YOLO actually
			# returns the center (x, y)-coordinates of the bounding
			# box followed by the boxes' width and height
			box = detection[0:4] * np.array([W, H, W, H])
			(centerX, centerY, width, height) = box.astype("int")

			# use the center (x, y)-coordinates to derive the top and
			# and left corner of the bounding box
			x = int(centerX - (width / 2))
			y = int(centerY - (height / 2))

			# update our list of bounding box coordinates, confidences,
			# and class IDs
			boxes.append([x, y, int(width), int(height)])
			confidences.append(float(confidence))
			classIDs.append(classID)

# apply non-maxima suppression to suppress weak, overlapping bounding
# boxes

In [20]:
idxs = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE_THRESHOLD,
	CONFIDENCE_THRESHOLD)


# ensure at least one detection exists

In [21]:
if len(idxs) > 0:
	# loop over the indexes we are keeping
	for i in idxs.flatten():
		# extract the bounding box coordinates
		(x, y) = (boxes[i][0], boxes[i][1])
		(w, h) = (boxes[i][2], boxes[i][3])

		color = [int(c) for c in COLORS[classIDs[i]]]

		cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
		text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
		cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
			0.5, color, 2)

In [22]:
cv2.imwrite("yash_1_output.png", image)

True